# Loading Libraries

In [ ]:
import concurrent.futures
import json
import numpy as np
import pandas as pd
import requests
import time
import urllib.request
import yfinance as yf
yf.pdr_override()

from datetime import datetime
import matplotlib.pyplot as plt
from pandas_datareader import data as pdr
from pandas.io.json import json_normalize
from tqdm import tqdm

# Variables

In [ ]:
today = datetime.today().strftime('%Y-%m-%d')

In [ ]:
def years_ago(dt, years):
    if dt.month == 2 and dt.day == 29:
        dt = dt.replace(day=28)

    return dt.replace(year=dt.year - years)

In [ ]:
one_year_ago = years_ago(datetime.today(), 1).strftime('%Y-%m-%d')

In [ ]:
ticker_list = "ticker_list.csv"

In [ ]:
# token='c3dg1raad3icrjj6ha80'
token='sandbox_c3dg1raad3icrjj6ha8g'
api_key='sandbox_c3dg1raad3icrjj6ha8g'
token='sandbox_c3dg1raad3icrjj6ha8g'
url = f'https://finnhub.io/api/v1/stock/symbol?exchange=US&token={api_key}'
hist_url = 'https://finnhub.io/api/v1/stock/metric?'

MAX_THREADS = 300

In [ ]:
print(url)

# Getting list_of_tickers

# Loading list_of_tickers

In [ ]:
df_tickers = pd.read_csv(ticker_list)
list_of_tickers = df_tickers['symbol'].values.tolist()

In [ ]:
print(len(list_of_tickers))

In [ ]:
count = 0
for ticker in list_of_tickers:
    print(ticker)
    count += 1
    if count >= 10:
        break

# Historical Price

# Case 1 - Threading

In [ ]:
d = {'ticker' : [], 'data' : []}
# d = {'link' : [], 'projects' : []}

############### SCRAPING PART ###############

# my scraping function that we are going to use for each item in profile
def scraper(ticker):
    global token
    data = requests.get(hist_url, params = {'symbol': ticker, 'token':token,'metric':'price'}, timeout=60)
    try:
        data=data.json()
    except json.decoder.JSONDecodeError:
        data = 'NA'
    return data

# def scraper(ticker):
#     global token
#     try:
#         r = requests.get(base_url, params = {'symbol': ticker, 'token':token,'metric':'price'}, timeout=60)
#         text = r.text
#         ret = json.loads(text)
#         df_hist = pd.DataFrame.from_dict(ret)
#         df_hist.reset_index(level=0, inplace=True) # # df_hist['index1'] = df_hist.index
#         df_hist = df_hist.pivot(index='symbol', columns=['index'], values='metric')
#         df_hist.reset_index(level=0, inplace=True)
#         return df_hist
#     except:
#         pass

In [ ]:
# # the threading part
# threads = min(MAX_THREADS, len(list_of_tickers))
# with concurrent.futures.ThreadPoolExecutor(max_workers=threads) as executor:
#     for number, future in tqdm(zip(list_of_tickers, executor.map(scraper, list_of_tickers, chunksize=100))):
#         if (future is None):
#             pass
#         else:
#             future.to_csv("stock_hist.csv", index=False, mode='a', header=False, date_format='%Y%m%d', encoding='utf-8-sig')

In [ ]:
# the threading part
result = []
threads = min(MAX_THREADS, len(list_of_tickers))
with concurrent.futures.ThreadPoolExecutor(max_workers=threads) as executor:
    for number, future in tqdm(zip(list_of_tickers, executor.map(scraper, list_of_tickers, chunksize=10))):
        if (future is None):
            pass
        else:
            result.append(future)

In [ ]:
print(len(result))

In [ ]:
print(result)

# Case 2 - For loop

In [35]:
def scraper(ticker):
    global token
    try:
        r = requests.get(base_url, params = {'symbol': ticker, 'token':token,'metric':'price'}, timeout=60)
        text = r.text
        ret = json.loads(text)
        df_hist = pd.DataFrame.from_dict(ret)
        df_hist.reset_index(level=0, inplace=True) # # df_hist['index1'] = df_hist.index
        df_hist = df_hist.pivot(index='symbol', columns=['index'], values='metric')
        df_hist.reset_index(level=0, inplace=True)
        return df_hist
    except:
        pass

In [ ]:
result = []
for ticker in tqdm(list_of_tickers):
    ret = scraper(ticker)
    result.append(ret)
    time.sleep(3)    

 10%|█████████████▉                                                                                                                             | 2763/27514 [4:22:23<70:09:15, 10.20s/it]

# Case 3 - Thread & CSV

In [ ]:
# def get_hist(list_of_tickers):
#     threads = min(MAX_THREADS, len(list_of_tickers))
#     with concurrent.futures.ThreadPoolExecutor(max_workers=threads) as executor:
#         for number, df in tqdm(zip(list_of_tickers, executor.map(get_hist_price, list_of_tickers, chunksize=500))):
#             if (df is None):
#                 pass
#             else:
#                 df.to_csv("stock_hist.csv", index=False, mode='a', header=False, date_format='%Y%m%d', encoding='utf-8-sig')

In [ ]:
# def get_hist_price(ticker)->dict:
#     global token
#     try:
#         r = requests.get(base_url, params = {'symbol': ticker, 'token':token,'metric':'price'}, timeout=60)
#         text = r.text
#         ret = json.loads(text)
#         df_hist = pd.DataFrame.from_dict(ret)
#         df_hist.reset_index(level=0, inplace=True) # # df_hist['index1'] = df_hist.index
#         df_hist = df_hist.pivot(index='symbol', columns=['index'], values='metric')
#         df_hist.reset_index(level=0, inplace=True)
#         return df_hist
#     except:
#         pass

In [ ]:
# def get_hist(list_of_tickers):
#     threads = min(MAX_THREADS, len(list_of_tickers))
#     with concurrent.futures.ThreadPoolExecutor(max_workers=threads) as executor:
#         for number, df in tqdm(zip(list_of_tickers, executor.map(get_hist_price, list_of_tickers, chunksize=500))):
#             if (df is None):
#                 pass
#             else:
#                 df.to_csv("stock_hist.csv", index=False, mode='a', header=False, date_format='%Y%m%d', encoding='utf-8-sig')

In [ ]:
# threads = min(MAX_THREADS, len(list_of_tickers))

In [ ]:
# get_hist(list_of_tickers)

## Checking

In [ ]:
r = requests.get(base_url, params = {'symbol': 'WSO', 'token':token,'metric':'price'})
text = r.text

In [ ]:
print(text)

# Target Price

In [ ]:
def get_target_price(ticker, token)->dict:
    base_url = 'https://finnhub.io/api/v1/stock/price-target?'
    r = requests.get(base_url, params = {'symbol': ticker, 'token':token})
    text = r.text
    text = json.loads(text)
    return text

In [ ]:
list_of_tickers[0:10]

In [ ]:
count = 0
records_of_tickers = []
for ticker in list_of_tickers:
    ret = get_target_price(ticker, token)
    records_of_tickers.append(ret)
    count += 1
    if count >= 10:
        break

In [ ]:
df_esti = pd.DataFrame(records_of_tickers)

In [ ]:
df_esti

In [ ]:
merged_df = df_esti.merge(df_hist, how = 'left', on = ['symbol'])

In [ ]:
merged_df

In [ ]:
merged_df.dropna(subset=['symbol'])

In [ ]:
print(type(merged_df['symbol'].iloc[0]))

In [ ]:
df1 = merged_df.replace(np.nan, '', regex=True)

In [ ]:
merged_df['symbol'] = merged_df['symbol'].replace('', np.nan)

In [ ]:
merged_df

In [ ]:
merged_df.dropna(subset=['symbol'], inplace=True)

In [ ]:
merged_df

# Recommendation Trends

In [ ]:
def get_recommmand_trends(ticker, token)->dict:
    base_url = 'https://finnhub.io/api/v1/stock/recommendation?'
    r = requests.get(base_url, params = {'symbol': ticker, 'token':token})
    text = r.text
    return (ticker, text)

In [ ]:
list_of_tickers = merged_df['symbol'].tolist()

In [ ]:
count = 0
records_of_tickers = []
for ticker in list_of_tickers:
    ret = get_recommmand_trends(ticker, token)
    records_of_tickers.append(ret)
    count += 1
    if count >= 10:
        break

In [ ]:
records_of_tickers

In [ ]:
df_recom = pd.DataFrame(records_of_tickers)

In [ ]:
df_recom

In [ ]:
df_recom.columns = ['symbol','recommendation']

In [ ]:
df_recom

In [ ]:
merged_df = merged_df.merge(df_recom, how = 'left', on = ['symbol'])

In [ ]:
merged_df